In [4]:
import numpy as np

**Goal:** Reach the bottom right cell of the grid starting from top left cell

**Note:** There are barriers in some of the cells so it might not always be possible to reach the goal

**Reward:** 10 for reaching goal; 0 otherwise

**Discount factor:** 0.9

**Note:** Agent should try to reach goal ASAP otherwise the overall reward will get diluted

In [5]:
class MazeSolver:

    def __init__(self, size, barrier_prob):
        self.size = size # Size of square grid
        self.barrier_prob = barrier_prob # Probability of a cell being a barrier
        self.maze = np.zeros((size, size))  # 0 represents an empty cell
        self.generate_maze()

        # Start state: top left cell
        self.start_state = (0, 0)
        # Terminal state: bottom right cell
        self.terminal_state = (size - 1, size - 1)

    def generate_maze(self):
        for i in range(self.size):
            for j in range(self.size):
                if np.random.rand() < self.barrier_prob:
                    self.maze[i, j] = 1  # Barrier is represented using 1
        self.maze[self.size-1, self.size-1] = 0  # This makes sure that terminal state is not a barrier
        self.maze[0, 0] = 0  # This makes sure that start state is not a barrier
        global my_maze
        my_maze = self.maze
        print("\nInitial maze with barriers:\n")
        self.print_initial_maze()

    def is_solvable(self):
        visited = set()

        def dfs(x, y):
            if not (0 <= x < self.size and 0 <= y < self.size) or my_maze[x, y] == 1 or (x, y) in visited:
                return False

            visited.add((x, y))

            if (x, y) == self.terminal_state:
                return True

            directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

            for dx, dy in directions:
                if dfs(x + dx, y + dy):
                    return True

            return False

        return dfs(*self.start_state)

    def is_valid_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size and self.maze[x, y] == 0

    def value_iteration(self, discount_factor=0.9, theta=0.3, max_iterations=1000):
        value_function = np.zeros((self.size, self.size))
        for iteration in range(max_iterations):
            print(f"\nValue Iteration - Iteration {iteration + 1}:\n")

            delta = 0
            for i in range(self.size):
                for j in range(self.size):
                    # If barrier skip
                    if self.maze[i, j] == 1:
                        continue
                    v = value_function[i, j]
                    # Value Iteration updates based on 1 step lookahed
                    value_function[i, j] = self.calculate_max_value(i, j, value_function, discount_factor)
                    delta = max(delta, abs(v - value_function[i, j]))

            self.print_maze(value_function)

            if delta < theta:
                print('\n=========================================\n')
                return value_function

    def calculate_max_value(self, x, y, value_function, discount_factor):
        max_value = float('-inf')
        # Obtain the maximum value from the 4 actions
        for action in range(4):
            next_x, next_y = self.get_next_position(x, y, action)
            if self.is_valid_move(next_x, next_y):
                if x == self.size-1 and y == self.size-1:
                    reward = 10
                else:
                    reward = 0

                max_value = max(max_value,reward + discount_factor * value_function[next_x, next_y])

        return max_value


    def get_next_position(self, x, y, action):
        if action == 0:  # Up
            return x - 1, y
        elif action == 1:  # Right
            return x, y + 1
        elif action == 2:  # Down
            return x + 1, y
        elif action == 3:  # Left
            return x, y - 1

    def print_maze(self, values):
        for i in range(self.size):
            for j in range(self.size):
                cell = "S" if (i, j) == self.start_state else "B" if self.maze[i, j] == 1 else f"{values[i, j]:.2f}"
                print(f"{cell:6} |", end="")
            print()

    def print_initial_maze(self):
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == (0,0):
                    print("S    |", end="")
                elif my_maze[i, j] == 1:
                    print("B    |", end="")
                else:
                    print(f"{my_maze[i, j]:.2f} |", end="")
            print()

    def getCommands(self):
        # Value iteration is called for solving the maze
        values = self.value_iteration()
        commands = []
        i,j=0,0
        nexti,nextj = 0,0
        while i!=self.size-1 or j!=self.size-1:
            max = float('-inf')
            maxDir = None
            nexti=i
            nextj=j
            if i-1>0 and max<values[i-1][j]:
                max = values[i-1][j]
                maxDir='u'
                nexti=i-1
            if i+1<self.size and max<values[i+1][j]:
                max = values[i+1][j]
                maxDir='d'
                nexti=i+1
            if j-1>0 and max<values[i][j-1]:
                max = values[i][j-1]
                maxDir='l'
                nextj=j-1
                nexti=i
            if j+1<self.size and max<values[i][j+1]:
                max = values[i][j+1]
                maxDir='r'
                nextj=j+1
                nexti=i

            if maxDir is not None:
                commands.append(maxDir)
                i=nexti
                j=nextj
        return commands

In [8]:
size = int(input("Enter the maze size: "))
barrier_prob = float(input("Enter the barrier probability: "))
maze_solver = MazeSolver(size, barrier_prob)
solvable = maze_solver.is_solvable()

if(solvable):
  optimal_path = maze_solver.getCommands()
  print("Path to goal:")
  print(optimal_path)
  print(f"Cost of path: {len(optimal_path)}")
else:
  print("Too many barriers. Solution does not exist.")

Enter the maze size: 4
Enter the barrier probability: 0.2

Initial maze with barriers:

S    |0.00 |B    |0.00 |
0.00 |0.00 |0.00 |0.00 |
0.00 |0.00 |0.00 |0.00 |
0.00 |B    |0.00 |0.00 |

Value Iteration - Iteration 1:

S      |0.00   |B      |0.00   |
0.00   |0.00   |0.00   |0.00   |
0.00   |0.00   |0.00   |0.00   |
0.00   |B      |0.00   |10.00  |

Value Iteration - Iteration 2:

S      |0.00   |B      |0.00   |
0.00   |0.00   |0.00   |0.00   |
0.00   |0.00   |0.00   |9.00   |
0.00   |B      |9.00   |18.10  |

Value Iteration - Iteration 3:

S      |0.00   |B      |0.00   |
0.00   |0.00   |0.00   |8.10   |
0.00   |0.00   |8.10   |16.29  |
0.00   |B      |16.29  |24.66  |

Value Iteration - Iteration 4:

S      |0.00   |B      |7.29   |
0.00   |0.00   |7.29   |14.66  |
0.00   |7.29   |14.66  |22.19  |
0.00   |B      |22.19  |29.98  |

Value Iteration - Iteration 5:

S      |0.00   |B      |13.19  |
0.00   |6.56   |13.19  |19.98  |
6.56   |13.19  |19.98  |26.98  |
5.90   |B      |26.9

**Observation 1:** Multiple optimal policies but unique optimal value function

**Observation 2:** Length of optimal path = 2 * (maze_size - 1)